## Set Environment

Before everything, **Edit-Notebook settings-GPU-save**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/5wFJbMW-vOmhBAh_z1hT-sLCBoml7p0BENZWv6oJRTndXeHdy18cazQ


In [ ]:
%cd '/content/gdrive/My Drive/'

/content/gdrive/My Drive


### Part 1 (Can be skipped!!!!)
If you already has **your own** ssh public and private key pair, then **skip** this part

In [ ]:
!mkdir ssh_keys

In [ ]:
!ls -a

set_environment.ipynb  .ssh


In [ ]:
!ssh-keygen

# follow this:
# /content/gdrive/My Drive/ssh_keys/id_rsa
# Enter
# Enter

Download SSH public key: /content/gdrive/My Drive/ssh_keys/id_rsa.pub

### Part 2 (Can be skipped!!!!)
If the key pair **does not exist** in github settings, go to github.com, log in to your account, go to settings-SSH and GPG keys, create a new SSH key, copy the key and paste.

### Part 3
**Copy your key pair to /root/ and Set git config**

In [ ]:
%cd '/root'
!mkdir '.ssh'

/root


In [ ]:
!cp -r '/content/gdrive/My Drive/ssh_keys/.' '/root/.ssh'

In [ ]:
%cd '/root/.ssh'
!ls -a

/root/.ssh
.  ..  id_rsa  id_rsa.pub


**Enter your own information here**

In [ ]:
# !git config --global user.email "addy@xyz.com"
# !git config --global user.name "your_name"

!git config --global user.email "mattcoldwater@gmail.com"
!git config --global user.name "mattcoldwater"

In [ ]:
!ssh -T -o StrictHostKeyChecking=no git@github.com

Hi mattcoldwater! You've successfully authenticated, but GitHub does not provide shell access.


### Part 4
run this block to commit and push (change commit comment!)

connect：git remote add origin(can be changed) branch_Name(default:main) url

git remote -v to check

nomrally pull first：git pull origin master

specially: git merge origin/master --allow-unrelated-histories

Then you need to deal with some conflicts

push：git push -u origin master

In [ ]:
%cd '/content/gdrive/My Drive/CSE527-Project/RNN_visualization/'
!ls

/content/gdrive/My Drive/CSE527-Project/RNN_visualization
cuda_IndRNN_onlyrecurrent.py  Indrnn_action_train.py   __pycache__
data_reader_ntu.py	      IndRNN_onlyrecurrent.py  README.md
Indrnn_action_network.py      opts.py		       skeleton_to_numpy.py


In [ ]:
# !git remote add origin master
!git remote -v

origin	git@github.com:mattcoldwater/RNN_visualization.git (fetch)
origin	git@github.com:mattcoldwater/RNN_visualization.git (push)


In [ ]:
!git add .

In [ ]:
!git commit -m "add gitinore"

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [ ]:
!git pull origin main

From github.com:mattcoldwater/RNN_visualization
 * branch            main       -> FETCH_HEAD
Already up to date.


In [ ]:
!git push -u origin main

Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 294 bytes | 147.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:mattcoldwater/RNN_visualization.git
   d20b170..5e6412c  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


## Demo

### preparation

In [ ]:
%cd '/content/gdrive/My Drive/CSE527-Project/RNN_visualization'
!ls -a

/content/gdrive/My Drive/CSE527-Project/RNN_visualization
cuda_IndRNN_onlyrecurrent.py  .gitignore		opts.py
Data			      Indrnn_action_network.py	__pycache__
data_reader_ntu.py	      Indrnn_action_train.py	README.md
.git			      IndRNN_onlyrecurrent.py	skeleton_to_numpy.py


cuda_IndRNN_onlyrecurrent.py  Indrnn_action_train.py   IndRNN_onlyrecurrent.py
Indrnn_action_network.py

In [ ]:
!cat README.md

### import packages

In [ ]:
import sys
import argparse
import os
import time
import numpy as np
import copy
import math
import csv
import glob
# import opts 
from __future__ import division
from __future__ import print_function

# from cupy.cuda import function
# from pynvrtc.compiler import Program
# from collections import namedtuple

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn import Parameter
from torch.autograd import Function
import torch.nn.init as weight_init
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack

### get numpy skeleton data

In [ ]:
## to do

### RNN_onlyrecurrent

In [ ]:
class IndRNNCell_onlyrecurrent(nn.Module):
    r"""An IndRNN cell with ReLU non-linearity. This is only the recurrent part where the input is already processed with w_{ih} * x + b_{ih}.

    .. math::
        input=w_{ih} * x + b_{ih}
        h' = \relu(input +  w_{hh} (*) h)
    With (*) being element-wise vector multiplication.

    Args:
        hidden_size: The number of features in the hidden state h

    Inputs: input, hidden
        - **input** (batch, input_size): tensor containing input features
        - **hidden** (batch, hidden_size): tensor containing the initial hidden
          state for each element in the batch.

    Outputs: h'
        - **h'** (batch, hidden_size): tensor containing the next hidden state
          for each element in the batch
    """

    def __init__(self, hidden_size, 
                 hidden_max_abs=None, recurrent_init=None):
        super(IndRNNCell_onlyrecurrent, self).__init__()
        self.hidden_size = hidden_size
        self.recurrent_init = recurrent_init
        self.weight_hh = Parameter(torch.Tensor(hidden_size))            
        self.reset_parameters()

    def reset_parameters(self):
        for name, weight in self.named_parameters():
            if "weight_hh" in name:
                if self.recurrent_init is None:
                    nn.init.uniform(weight, a=0, b=1)
                else:
                    self.recurrent_init(weight)

    def forward(self, input, hx):
        return F.relu(input + hx * self.weight_hh.unsqueeze(0).expand(hx.size(0), len(self.weight_hh)))


class IndRNN_onlyrecurrent(nn.Module):
    r"""Applies an IndRNN with `ReLU` non-linearity to an input sequence. 
    This is only the recurrent part where the input is already processed with w_{ih} * x + b_{ih}.


    For each element in the input sequence, each layer computes the following
    function:

    .. math::

        h_t = \relu(input_t +  w_{hh} (*) h_{(t-1)})

    where :math:`h_t` is the hidden state at time `t`, and :math:`input_t`
    is the input at time `t`. (*) is element-wise multiplication.

    Args:
        hidden_size: The number of features in the hidden state `h`
        batch_first: If ``True``, then the input and output tensors are provided
            as `(batch, seq, feature)`

    Inputs: input, h_0
        - **input** of shape `(seq_len, batch, input_size)`: tensor containing the features
          of the input sequence. The input can also be a packed variable length
          sequence. See :func:`torch.nn.utils.rnn.pack_padded_sequence`
          or :func:`torch.nn.utils.rnn.pack_sequence`
          for details.
        - **h_0** of shape `(num_directions, batch, hidden_size)`: tensor
          containing the initial hidden state for each element in the batch.
          Defaults to zero if not provided.

    Outputs: output, h_n
        - **output** of shape `(seq_len, batch, hidden_size * num_directions)`: tensor
          containing the output features (`h_k`) from the last layer of the RNN,
          for each `k`.  If a :class:`torch.nn.utils.rnn.PackedSequence` has
          been given as the input, the output will also be a packed sequence.
        - **h_n** (num_directions, batch, hidden_size): tensor
          containing the hidden state for `k = seq_len`.
    """

    def __init__(self, hidden_size, 
                 batch_first=False, bidirectional=False, recurrent_inits=None,
                 **kwargs):
        super(IndRNN_onlyrecurrent, self).__init__()
        self.hidden_size = hidden_size
        self.batch_first = batch_first
        self.bidirectional = bidirectional

        num_directions = 2 if self.bidirectional else 1

        if batch_first:
            self.time_index = 1
            self.batch_index = 0
        else:
            self.time_index = 0
            self.batch_index = 1

        cells = []
        directions = []
        if recurrent_inits is not None:
            kwargs["recurrent_init"] = recurrent_inits
        for dir in range(num_directions):
            directions.append(IndRNNCell_onlyrecurrent(hidden_size, **kwargs))
        self.cells = nn.ModuleList(directions)

        h0 = torch.zeros(hidden_size * num_directions)
        self.register_buffer('h0', torch.autograd.Variable(h0))

    def forward(self, x, hidden=None):
        time_index = self.time_index
        batch_index = self.batch_index
        num_directions = 2 if self.bidirectional else 1
        hidden_init = self.h0.unsqueeze(0).expand(
            x.size(batch_index),
            self.hidden_size * num_directions).contiguous()

        x_n = []
        for dir, cell in enumerate(self.cells):
            hx_cell = hidden_init[
                :, self.hidden_size * dir: self.hidden_size * (dir + 1)]
            outputs = []
            hiddens = []
            x_T = torch.unbind(x, time_index)
            if dir == 1:
                x_T = reversed(x_T)
            for x_t in x_T:
                hx_cell = cell(x_t, hx_cell)
                outputs.append(hx_cell)
            if dir == 1:
                outputs = outputs[::-1]
            x_cell = torch.stack(outputs, time_index)
            x_n.append(x_cell)
            hiddens.append(hx_cell)
        x = torch.cat(x_n, -1)
        return x.squeeze(2), torch.cat(hiddens, -1)

### complete RNN

In [ ]:
class Batch_norm_step(nn.Module):
    def __init__(self,  hidden_size,seq_len):
        super(Batch_norm_step, self).__init__()
        self.hidden_size = hidden_size
        
        self.max_time_step=seq_len
        self.bn = nn.BatchNorm1d(hidden_size) 

    def forward(self, x):
        x=x.permute(1,2,0)
        x= self.bn(x.clone())
        x=x.permute(2,0,1)
        return x
class Dropout_overtime(torch.autograd.Function):
  @staticmethod
  def forward(ctx, input, p=0.5,training=False):
    output = input.clone()
    noise = input.data.new(input.size(-2),input.size(-1))  #torch.ones_like(input[0])
    if training:            
      noise.bernoulli_(1 - p).div_(1 - p)
      noise = noise.unsqueeze(0).expand_as(input)
      output.mul_(noise)
    ctx.save_for_backward(noise)
    ctx.training=training
    return output
  @staticmethod
  def backward(ctx, grad_output):
    noise,=ctx.saved_tensors
    if ctx.training:
      return grad_output.mul(noise),None,None
    else:
      return grad_output,None,None
dropout_overtime=Dropout_overtime.apply

import argparse
import opts     
parser = argparse.ArgumentParser(description='pytorch action')
opts.train_opts(parser)
args = parser.parse_args()
MAG=args.MAG
U_bound=np.power(10,(np.log10(MAG)/args.seq_len))
U_lowbound=np.power(10,(np.log10(1.0/MAG)/args.seq_len))  
  
class stackedIndRNN_encoder(nn.Module):
    def __init__(self, input_size, outputclass, isCuda):
        super(stackedIndRNN_encoder, self).__init__()        
        hidden_size=args.hidden_size
        
        self.isCuda = isCuda
            
        self.DIs=nn.ModuleList()
        denseinput=nn.Linear(input_size*3, hidden_size, bias=True)
        self.DIs.append(denseinput)
        for x in range(args.num_layers - 1):
            denseinput = nn.Linear(hidden_size, hidden_size, bias=True)
            self.DIs.append(denseinput)                
        
        self.BNs = nn.ModuleList()
        for x in range(args.num_layers):
            bn = Batch_norm_step(hidden_size,args.seq_len)
            self.BNs.append(bn)                      
  
        self.RNNs = nn.ModuleList()
        rnn = IndRNN(hidden_size=hidden_size) #IndRNN
        self.RNNs.append(rnn)  
        for x in range(args.num_layers-1):
            rnn = IndRNN(hidden_size=hidden_size) #IndRNN
            self.RNNs.append(rnn)         
            
        self.lastfc = nn.Linear(hidden_size, outputclass, bias=True)
        self.init_weights()

    def init_weights(self):
      for name, param in self.named_parameters():
        if 'weight_hh' in name:
          param.data.uniform_(0,U_bound)          
        if 'RNNs.'+str(args.num_layers-1)+'.weight_hh' in name:
          param.data.uniform_(U_lowbound,U_bound)    
        if 'DIs' in name and 'weight' in name:
          param.data.uniform_(-args.ini_in2hid,args.ini_in2hid)               
        if 'bns' in name and 'weight' in name:
          param.data.fill_(1)      
        if 'bias' in name:
          param.data.fill_(0.0)              
    def forward(self, input):
        all_output = []
        rnnoutputs={}
        hidden_x={}               
        seq_len, batch_size, indim,_=input.size()
        #print(seq_len, batch_size, indim)
             
        input=input.view(seq_len,batch_size,3*indim)                  
        #print(input.shape)
        for x in range(1,len(self.RNNs)+1):
          if self.isCuda:
              hidden_x['hidden%d'%x]=Variable(torch.zeros(1,batch_size,args.hidden_size).cuda())
          else:
              hidden_x['hidden%d'%x]=Variable(torch.zeros(1,batch_size,args.hidden_size))
                            
        rnnoutputs['rnnlayer0']=input
        for x in range(1,len(self.RNNs)+1):
          rnnoutputs['rnnlayer%d'%(x-1)]=rnnoutputs['rnnlayer%d'%(x-1)].contiguous().view(seq_len*batch_size,-1)
          rnnoutputs['rnnlayer%d'%(x-1)]=self.DIs[x-1](rnnoutputs['rnnlayer%d'%(x-1)])   
          rnnoutputs['rnnlayer%d'%(x-1)]=rnnoutputs['rnnlayer%d'%(x-1)].view(seq_len,batch_size,-1)  
          #rnnoutputs['rnnlayer%d'%x]= self.RNNs[x-1](rnnoutputs['rnnlayer%d'%(x-1)], hidden_x['hidden%d'%x])        
          if self.isCuda:
              rnn_x_out = self.RNNs[x-1](rnnoutputs['rnnlayer%d'%(x-1)], hidden_x['hidden%d'%x])
          else:
              rnn_x_out, rnn_h_out= self.RNNs[x-1](rnnoutputs['rnnlayer%d'%(x-1)], hidden_x['hidden%d'%x])
              hidden_x['hidden%d'%x] = rnn_h_out

          rnnoutputs['rnnlayer%d'%x] = rnn_x_out

          rnnoutputs['rnnlayer%d'%x]=self.BNs[x-1](rnnoutputs['rnnlayer%d'%x])     
          if args.dropout>0:
            rnnoutputs['rnnlayer%d'%x]= dropout_overtime(rnnoutputs['rnnlayer%d'%x],args.dropout,self.training) 
        temp=rnnoutputs['rnnlayer%d'%len(self.RNNs)][-1]
        output = self.lastfc(temp)
        return output

### train

In [ ]:
# Set the random seed manually for reproducibility.
seed=100
torch.manual_seed(seed)
CUDA = False
if torch.cuda.is_available():
  torch.cuda.manual_seed(seed)
  CUDA = True
else:
  print("WARNING: CUDA not available")
  CUDA = False

import opts     
parser = argparse.ArgumentParser(description='pytorch action')
opts.train_opts(parser)
args = parser.parse_args()
print(args)

import Indrnn_action_network

batch_size = args.batch_size
print('BATCH_SIZE', batch_size)
seq_len=args.seq_len
outputclass=60
indim=50
gradientclip_value=10
U_bound=Indrnn_action_network.U_bound



model = Indrnn_action_network.stackedIndRNN_encoder(indim, outputclass, CUDA)  
if CUDA:
    model.cuda()
criterion = nn.CrossEntropyLoss()

#Adam with lr 2e-4 works fine.
learning_rate=args.lr
if args.use_weightdecay_nohiddenW:
  param_decay=[]
  param_nodecay=[]
  for name, param in model.named_parameters():
    if 'weight_hh' in name or 'bias' in name:
      param_nodecay.append(param)      
      #print('parameters no weight decay: ',name)          
    else:
      param_decay.append(param)      
      #print('parameters with weight decay: ',name)          

  if args.opti=='sgd':
    optimizer = torch.optim.SGD([
            {'params': param_nodecay},
            {'params': param_decay, 'weight_decay': args.decayfactor}
        ], lr=learning_rate,momentum=0.9,nesterov=True)   
  else:                
    optimizer = torch.optim.Adam([
            {'params': param_nodecay},
            {'params': param_decay, 'weight_decay': args.decayfactor}
        ], lr=learning_rate) 
else:  
  if args.opti=='sgd':   
    optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate,momentum=0.9,nesterov=True)
  else:                      
    optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

if args.test_CV:
  train_datasets='train_CV_ntus'
  test_dataset='test_CV_ntus'
else:
  train_datasets='train_ntus'
  test_dataset='test_ntus'
  
#from data_reader_numpy_witheval import DataHandler_train,DataHandler_eval  
#from data_reader_numpy_test import DataHandler as testDataHandler

from data_reader_ntu import initialize_data_handlers

dh_train, dh_eval, dh_test = initialize_data_handlers(batch_size, seq_len)

#dh_train = DataHandler_train(batch_size,seq_len)
#dh_eval = DataHandler_eval(batch_size,seq_len)
#dh_test= testDataHandler(batch_size,seq_len)

num_train_batches=int(np.ceil(dh_train.GetDatasetSize()/(batch_size+0.0)))
num_eval_batches=int(np.ceil(dh_eval.GetDatasetSize()/(batch_size+0.0)))
num_test_batches=int(np.ceil(dh_test.GetDatasetSize()/(batch_size+0.0)))


def train(num_train_batches):
  model.train()
  tacc=0
  count=0
  start_time = time.time()
  for batchi in range(0,num_train_batches):
    inputs,targets=dh_train.GetBatch()
    inputs=inputs.transpose(1,0,2,3)
    
    if CUDA:
        inputs=Variable(torch.from_numpy(inputs).cuda(), requires_grad=True).float()
        targets=Variable(torch.from_numpy(np.int64(targets)).cuda(), requires_grad=False)
    else:
        inputs=Variable(torch.from_numpy(inputs), requires_grad=True).float()
        targets=Variable(torch.from_numpy(np.int64(targets)), requires_grad=False)

    model.zero_grad()
    if args.constrain_U:
      clip_weight(model,U_bound)
    output=model(inputs)
    loss = criterion(output, targets)

    pred = output.data.max(1)[1] # get the index of the max log-probability
    accuracy = pred.eq(targets.data).cpu().sum().numpy()/(0.0+targets.size(0))      
          
    loss.backward()
    clip_gradient(model,gradientclip_value)
    optimizer.step()
    
    tacc=tacc+accuracy#loss.data.cpu().numpy()#accuracy
    count+=1
  elapsed = time.time() - start_time
  print ("training accuracy: ", tacc/(count+0.0)  )
  #print ('time per batch: ', elapsed/num_train_batches)
  
def set_bn_train(m):
    classname = m.__class__.__name__
    if classname.find('BatchNorm') != -1:
      m.train()       
def eval(dh,num_batches,use_bn_trainstat=False):
  model.eval()
  if use_bn_trainstat:
    model.apply(set_bn_train)
  tacc=0
  count=0  
  start_time = time.time()
  while(1):  
    inputs,targets=dh.GetBatch()
    inputs=inputs.transpose(1,0,2,3)
    if CUDA:
        inputs=Variable(torch.from_numpy(inputs).cuda()).float()
        targets=Variable(torch.from_numpy(np.int64(targets)).cuda())
    else: 
        inputs=Variable(torch.from_numpy(inputs)).float()
        targets=Variable(torch.from_numpy(np.int64(targets)))
    output=model(inputs)
    pred = output.data.max(1)[1] # get the index of the max log-probability
    accuracy = pred.eq(targets.data).cpu().sum().numpy()        
    tacc+=accuracy
    count+=1
    if count==num_batches*args.eval_fold:
      break
  elapsed = time.time() - start_time
  print ("eval accuracy: ", tacc/(count*targets.data.size(0)+0.0)  )
  #print ('eval time per batch: ', elapsed/(count+0.0))
  return tacc/(count*targets.data.size(0)+0.0)


def test(dh,num_batches,use_bn_trainstat=False):
  model.eval()
  if use_bn_trainstat:
    model.apply(set_bn_train)
  tacc=0
  count=0  
  start_time = time.time()
  total_testdata=dh_test.GetDatasetSize()  
  total_ave_acc = None
  test_labels = None
  while(1):  
    inputs,targets=dh.GetBatch()
    inputs=inputs.transpose(1,0,2,3)
    if test_labels is None:
        test_labels = targets
    else:
        test_labels=np.concatenate(
                [test_labels, targets], 
                axis=0,
                )
    if CUDA:
        inputs=Variable(torch.from_numpy(inputs).cuda()).float()
        targets=Variable(torch.from_numpy(np.int64(targets)).cuda())
    else:
        inputs=Variable(torch.from_numpy(inputs)).float()
        targets=Variable(torch.from_numpy(np.int64(targets)))
        
    output=model(inputs)
    pred = output.data.max(1)[1] # get the index of the max log-probability
    accuracy = pred.eq(targets.data).cpu().sum().numpy()    
    
    if not total_ave_acc is None :
        total_ave_acc = np.concatenate(
                [total_ave_acc, output.data.cpu().numpy()]
                , axis=0,
                )
    else:
        total_ave_acc = output.data.cpu().numpy()
    
    tacc+=accuracy
    count+=1
    if count==num_batches:
      break    
  top = np.argmax(total_ave_acc, axis=-1)
  eval_acc=np.mean(np.equal(top, test_labels))    
  elapsed = time.time() - start_time
  print ("test accuracy: ",  eval_acc)
  #print ('test time per batch: ', elapsed/(count+0.0))
  return eval_acc

def clip_gradient(model, clip):
    for p in model.parameters():
        p.grad.data.clamp_(-clip,clip)
        #print(p.size(),p.grad.data)

def adjust_learning_rate(optimizer, lr):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr     

def clip_weight(RNNmodel, clip):
    for name, param in RNNmodel.named_parameters():
      if 'weight_hh' in name:
        param.data.clamp_(-clip,clip)
    
lastacc=0
dispFreq=20
patience=0
reduced=1
for batchi in range(1,10000000):
  for i in range(num_train_batches//dispFreq):

    train(dispFreq)
  test_acc=eval(dh_eval,num_eval_batches,args.use_bneval)
  
  if (test_acc >lastacc):
    model_clone = copy.deepcopy(model.state_dict())   
    opti_clone = copy.deepcopy(optimizer.state_dict()) 
    lastacc=test_acc
    patience=0
  elif patience>int(args.pThre/reduced+0.5):
    reduced=reduced*2
    print ('learning rate',learning_rate)
    model.load_state_dict(model_clone)
    optimizer.load_state_dict(opti_clone)
    patience=0
    learning_rate=learning_rate*0.1
    adjust_learning_rate(optimizer,learning_rate)     
    if learning_rate<args.end_rate:
      break  
    test_acc=test(dh_test,num_test_batches)     
 
  else:
    patience+=1 
    
test_acc=test(dh_test,num_test_batches)  
test_acc=test(dh_test,num_test_batches,True)     
save_name='indrnn_action_model' 
with open(save_name, 'wb') as f:
    torch.save(model, f)